In [2]:
import polars as pl
import polars.selectors as cs

In [3]:
disasters = pl.read_csv("1970-2021_DISASTERS.csv", ignore_errors=True).rename(
        {"Disaster Type" : "disasterType"})

disasters.head()


Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,disasterType,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,f64,f64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,f64,i64,str,i64,str
"""1970-0013-ARG""",1970,13,null,"""Natural""","""Hydrological""","""Flood""",null,null,null,"""Argentina""","""ARG""","""South America""","""Americas""","""Mendoza""",null,null,null,null,null,null,null,null,"""Km2""",null,null,null,null,1970,1,4,1970,1,4,36,null,null,null,null,null,null,25000,15.001282,null,null,null,null
"""1970-0109-AUS""",1970,109,null,"""Natural""","""Meteorological""","""Storm""","""Tropical cyclone""",null,"""Ada""","""Australia""","""AUS""","""Australia and New Zealand""","""Oceania""","""Queensland""",null,null,null,null,null,null,null,null,"""Kph""",null,null,null,null,1970,1,null,1970,1,null,13,null,null,null,null,null,null,72475,15.001282,null,null,null,null
"""1970-0044-BEN""",1970,44,null,"""Natural""","""Hydrological""","""Flood""",null,null,null,"""Benin""","""BEN""","""Western Africa""","""Africa""","""Atacora region""",null,null,null,"""Yes""",null,null,null,null,"""Km2""",null,null,null,null,1970,9,null,1970,9,null,null,null,null,null,null,null,null,200,15.001282,null,null,null,null
"""1970-0063-BGD""",1970,63,null,"""Natural""","""Meteorological""","""Storm""","""Tropical cyclone""",null,null,"""Bangladesh""","""BGD""","""Southern Asia""","""Asia""","""Khulna, Chittagong""",null,null,null,"""Yes""",null,null,null,null,"""Kph""",null,null,null,null,1970,11,12,1970,11,12,300000,null,3648000,null,3648000,null,null,86400,15.001282,null,null,null,null
"""1970-0026-BGD""",1970,26,null,"""Natural""","""Meteorological""","""Storm""",null,null,null,"""Bangladesh""","""BGD""","""Southern Asia""","""Asia""",null,null,null,null,null,null,null,null,null,"""Kph""",null,null,null,null,1970,4,13,1970,4,13,17,null,110,null,110,null,null,null,15.001282,null,null,null,null


In [4]:
annual_country_freq = disasters.select(
    ["Year", "disasterType", "Country"]
    ).group_by(["Year", "Country", "disasterType"], maintain_order=True).len()

# group least frequent disaster types into "other category"
least_freq = annual_country_freq.group_by("disasterType").sum().sort("len")[:6].select("disasterType")
least_freq = least_freq.to_series().to_list()
annual_country_freq = annual_country_freq.with_columns(disasterType = pl.col("disasterType").replace(least_freq, ["Other"] * 6 ))
annual_country_freq.filter(pl.col("disasterType") == "Other")


Year,Country,disasterType,len
i64,str,str,u32
1971,"""Canada""","""Other""",1
1975,"""Costa Rica""","""Other""",1
1975,"""Ecuador""","""Other""",2
1976,"""Costa Rica""","""Other""",1
1976,"""Ecuador""","""Other""",1
…,…,…,…
2021,"""Philippines (the)""","""Other""",1
2021,"""Canary Is""","""Other""",1
2019,"""Uganda""","""Other""",1


In [15]:
# consolidate least frequent disaster types into Other category
annual_frequencies = disasters.with_columns(\
    disasterType = pl.col("disasterType").replace(least_freq, ["Other"] * 6 ))

annual_frequencies = annual_frequencies.select(
    ["Year", "disasterType"]
    ).group_by(["Year", "disasterType"], maintain_order=True).len()

annual_frequencies = annual_frequencies.with_columns(pl.concat_str([pl.col("Year"), pl.col("disasterType")]).alias("compkey"))

annual_frequencies


Year,disasterType,len,compkey
i64,str,u32,str
1970,"""Flood""",31,"""1970Flood"""
1970,"""Storm""",24,"""1970Storm"""
1970,"""Drought""",2,"""1970Drought"""
1970,"""Landslide""",6,"""1970Landslide"""
1970,"""Earthquake""",12,"""1970Earthquake"""
…,…,…,…
2021,"""Landslide""",10,"""2021Landslide"""
2021,"""Earthquake""",19,"""2021Earthquake"""
2021,"""Other""",10,"""2021Other"""
